# Tổng quan phần thu thập dữ liệu
1. Mục tiêu
- Hiện nay có rất nhiều nhà tuyển dụng đăng thông tin tuyển dụng trên nhiều trang web như : https://vieclam24h.vn/, https://www.topcv.vn/viec-lam, https://careerbuilder.vn/tim-viec-lam.html, https://www.vieclamtot.com/viec-lam
- Do đó để dự đoán xu hướng công việc cũng như tìm hiểu thị trường việc làm, chúng ta sẽ chọn thu thập dữ liệu từ các trang web [Viêc làm 24h](https://vieclam24h.vn/).
3. Công cụ sử dụng
- Trong phần này, ta sử dụng những thư viện được python hỗ trợ như: request_html để gửi yêu cầu lấy trang web về.
- BeautifulSoup để phân tích cấu trúc của trang web. Sau đó lấy những trường cần thiết.
5. Các bước thu thập dữ liệu
- Ban đầu trang chủ trang web có một các thông tin về các công việc mới nhất, các công việc được đăng gần đây nhất. Ta sẽ lấy các link của các công việc này. Sau đó đến đường dẫn các công việc này để lấy thông tin chi tiết về công việc. Chúng em sẽ có 2 bước thu thập dữ liệu:
    - Step 1: Thu thập dữ liệu từ trang chủ sau đó lưu các đường link vào 'job_pool.csv'
    - Thu thập dữ liệu từ các trang chi tiết và lưu tất cả công việc vào một dataset có tên là 'raw_dataset.csv'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# import những  thư viện cần thiết
import pandas as pd
import numpy as np
!pip install requests_html
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import math
import requests
import time

## Hàm gửi yêu cầu đến server yêu cầu tải toàn bộ nội dung trang web

In [3]:

def get_content(url, headers):
    # time.sleep(1)
    session = HTMLSession()
    response = session.get(url, headers=headers)
    if(response.status_code == 200):
        print("Success!!!")
        return response
    return None

In [4]:
BASE_URL = 'https://vieclam24h.vn'


headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}


## Step 1: Lấy những liên kết của trang web công việc
- Đầu tiên, khi vô một trang web tìm việc, ta sẽ thấy một loạt danh sách các công việc, sau khi nhấn vào từng cái sẽ liên kết trực tiếp đến trang chi tiết công việc. Vậy đầu tiên ta đến trang tìm kiếm và lọc tất cả các đường dẫn đến công việc.

In [11]:
# Code demo request lấy page list công việc
job_pool = pd.DataFrame()
for work_method in range(1,7):
   #https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6
   url = BASE_URL + '/tim-kiem-viec-lam-nhanh?&working_method={}'.format(work_method)
   print(url)
   page_content = get_content(url, headers)
   soup = BeautifulSoup(page_content.text, 'html.parser')

   buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
   buffer2 = buffer1.find('span',class_="font-semibold").text
   n_jobs = int(buffer2.replace(',',''))
   # Trung bình mỗi page có 30 công việc
   n_pages = min(51, math.ceil(n_jobs/30) + 1)
   print('Có {} công việc, {} trang ở work_method = {}'.format(n_jobs, n_pages, work_method))


   for page in range(1,n_pages):
      # Khởi tạo url vầ yêu cầu lấy page về
      url = BASE_URL + '/tim-kiem-viec-lam-nhanh?page={}&working_method={}'.format(page, work_method)
      print(url)
      page_content = get_content(url, headers)
      soup = BeautifulSoup(page_content.text, 'html.parser')

      containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
      print('Đã tìm thấy {} công việc'.format(len(containers)))
      for container in containers:
         dict_job = {}

         buffer = BASE_URL + container.find('a', href=True)['href']
         dict_job['Liên kết'] = buffer if buffer else None

         buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
         dict_job['Tên công việc'] = buffer.text if buffer else None

         buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
         dict_job['Tên công ty'] = buffer.text if buffer else None

         buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
         dict_job['Mức lương'] = buffer.text if buffer else None

         buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
         dict_job['Khu vực tuyển'] = buffer.text if buffer else None

         job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool.sample(5)



https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?&working_method=1
Success!!!
Có 5554 công việc, 51 trang ở work_method = 1
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=1&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=3&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=4&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=5&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=6&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=7&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=8&working_method=1
Success!!!
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
629,https://vieclam24h.vn/ban-si-ban-le-quan-ly-cu...,Giám Sát Cửa Hàng Thời Trang,Công Ty TNHH Aroma Home,6.5 - 8 triệu,TP.HCM
383,https://vieclam24h.vn/ban-hang-kinh-doanh/nam-...,"Nam Nhân Viên Kinh Doanh (Không Chạy Doanh Số,...",Công Ty TNHH Vlxd Kiến Hoa (Long An),15 - 20 triệu,"TP.HCM, Long An, Đồng Nai"
721,https://vieclam24h.vn/luat-phap-ly-tuan-thu/th...,Thư Ký Nghiệp Vụ,Văn Phòng Công Chứng Tân Phú,5 - 7 triệu,TP.HCM
1365,https://vieclam24h.vn/cham-soc-khach-hang/nhan...,Nhân Viên Văn Phòng - Hỗ Trợ Và Chăm Sóc Khách...,Công Ty TNHH PHP Group International Việt Nam,10 - 20 triệu,TP.HCM
1020,https://vieclam24h.vn/tai-chinh-dau-tu-chung-k...,Nhân Viên Tư Vấn Tín Dụng - Kênh Trả Góp Tại Q...,Công ty Tài Chính TNHH HD SAISON,8 - 15 triệu,Quảng Ngãi


In [12]:
# Lưu dữ liệu vào file csv
job_pool.to_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/job_pool.csv', encoding='utf-8-sig', index=False)
# job_pool.to_csv('../data/job_pool.csv', encoding='utf-8-sig', index=False)

# Step 2: Thu thập dữ liệu của từng trang web
- Sau khi thu thập được các link trang web, ta tiến hành thu thập dữ liệu của từng trang web bằng cách sử dụng cách tương tự

In [13]:
# Đọc dữ liệu từ file job_links.csv
df = pd.read_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/job_pool.csv', encoding='utf-8-sig')
# df = pd.read_csv('../data/job_pool.csv')
df.sample(5)

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
1043,https://vieclam24h.vn/moi-truong-xu-ly-chat-th...,Kỹ Sư Môi Trường Mảng Xử Lý Nước Thải Tại Thủ Đức,Công Ty Cổ Phần Tổng Thầu Môi Trường King Power,8 - 15 triệu,TP.HCM
688,https://vieclam24h.vn/hanh-chinh-thu-ky/office...,Office Administrator (Admin Văn Phòng),Công Ty TNHH Đầu Tư Xây Dựng Aquarius,7 - 15 triệu,TP.HCM
1399,https://vieclam24h.vn/tai-chinh-dau-tu-chung-k...,"Buôn Hồ, Đắk Lắk - Nhân Viên Tư Vấn Tín Dụng (...",Công ty Tài Chính TNHH HD SAISON,8 - 15 triệu,Quảng Ngãi
1719,https://vieclam24h.vn/tu-van/sinh-vien-lam-the...,Sinh Viên Làm Thêm Công Việc Bán Cà Phê Tại Quầy,The Venus Shop,5 - 11 triệu,TP.HCM
1743,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Nhân Viên Lái Xe Khách Sạn Bằng D (Hotel Driver),Elegance Hospitality Group - Công Ty TNHH DL V...,11 - 14 triệu,TP.HCM


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1877 entries, 0 to 1876
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Liên kết       1877 non-null   object
 1   Tên công việc  1877 non-null   object
 2   Tên công ty    1877 non-null   object
 3   Mức lương      1877 non-null   object
 4   Khu vực tuyển  1877 non-null   object
dtypes: object(5)
memory usage: 73.4+ KB


In [15]:
raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển',
# Thông tin chung
'Thời gian thử việc','Cấp bậc', 'Yêu cầu giới tính','Số lượng tuyển', 'Hình thức làm việc','Độ tuổi', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])

for i, url in enumerate(df['Liên kết']):
    print("Công việc thứ {}".format(i))
    job_content = get_content(url, headers=headers)
    if (job_content != None):
      soup = BeautifulSoup(job_content.text, 'html.parser')

    dict_job = {}

    dict_job['Liên kết'] = url

    overview = soup.find('div', class_='md:ml-7 w-full')
    try:
        buffer = overview.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
        dict_job['Tên công ty'] = buffer.text if buffer else None
    except:
        dict_job['Tên công ty'] = None
    try:
        buffer = overview.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
        dict_job['Tên công việc'] = buffer.text if buffer else None
    except:
        dict_job['Tên công việc'] = None
    try:
        buffer = overview.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
        dict_job['Mức lương'] = buffer.text if buffer else None
    except:
        dict_job['Mức lương'] = None

    try:
        buffer = overview.find_all('a', class_ ="hover:text-se-accent")
        dict_job['Khu vực tuyển'] = ' '.join([element.text for element in buffer]) if buffer else None
    except:
        dict_job['Khu vực tuyển'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu giới tính")
        dict_job['Yêu cầu giới tính'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu giới tính'] = None

    try:
        buffer = soup.find('p', string="Cấp bậc")
        dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Cấp bậc'] = None

    try:
        buffer = soup.find('p', string="Thời gian thử việc")
        dict_job['Thời gian thử việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Thời gian thử việc'] = None

    try:
        buffer = soup.find('p', string="Số lượng tuyển")
        dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Số lượng tuyển'] = None


    try:
        buffer = soup.find('p', string="Hình thức làm việc")
        dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Hình thức làm việc'] = None

    try:
        buffer = soup.find('p', string="Độ tuổi")
        dict_job['Độ tuổi'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Độ tuổi'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu bằng cấp")
        dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu bằng cấp'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
        dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu kinh nghiệm'] = None

    try:
        buffer = soup.find('p', string="Ngành nghề")
        dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Ngành nghề'] = None

    try:
        buffer1 = soup.find('h4', string="Mô tả công việc")
        buffer2 = buffer1.find_next_sibling('div')
        dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None
    except:
        dict_job['Mô tả công việc'] = None

    try:
        buffer1 = soup.find('h4', string="Yêu cầu công việc")
        buffer2 = buffer1.find_next_sibling('div')
        dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None
    except:
        dict_job['Yêu cầu công việc'] = None

    try:
        buffer1 = soup.find('h4', string="Quyền lợi")
        buffer2 = buffer1.find_next_sibling('div')
        dict_job['Quyền lợi'] = buffer2.text if buffer2 else None
    except:
        dict_job['Quyền lợi'] = None

    try:
        buffer1 = soup.find('h4', string="Địa điểm làm việc")
        buffer2 = buffer1.find_next_sibling('ul')
        buffer3 = buffer2.find_all('li', class_ = "jsx-d84db6a84feb175e mb-2 flex text-14")
        dict_job['Địa điểm làm việc'] = '; '.join([element.text for element in buffer3 if element.text.strip()]) if buffer3 else None
    except:
        dict_job['Địa điểm làm việc'] = None

    try:
        buffer1 = soup.find('h3', string="Từ khoá")
        buffer2 = buffer1.find_next_sibling('div')
        dict_job['Từ khóa'] = buffer2.text if buffer2 else None
    except:
        dict_job['Từ khóa'] = None

    company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

    buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
    try:
        for buffer in buffers:
            if buffer.get_text() == 'Địa chỉ:':
                dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
            if buffer.get_text() == 'Quy mô:':
                dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None
    except:
        dict_job['Địa chỉ công ty'] = None
        dict_job['Quy mô công ty'] = None

    raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)
    raw_job_details

Công việc thứ 0
Success!!!
Công việc thứ 1
Success!!!
Công việc thứ 2
Success!!!
Công việc thứ 3
Success!!!
Công việc thứ 4
Success!!!
Công việc thứ 5
Success!!!
Công việc thứ 6
Success!!!
Công việc thứ 7
Success!!!
Công việc thứ 8
Success!!!
Công việc thứ 9
Success!!!
Công việc thứ 10
Success!!!
Công việc thứ 11
Success!!!
Công việc thứ 12
Success!!!
Công việc thứ 13
Success!!!
Công việc thứ 14
Success!!!
Công việc thứ 15
Success!!!
Công việc thứ 16
Success!!!
Công việc thứ 17
Success!!!
Công việc thứ 18
Success!!!
Công việc thứ 19
Success!!!
Công việc thứ 20
Success!!!
Công việc thứ 21
Success!!!
Công việc thứ 22
Success!!!
Công việc thứ 23
Success!!!
Công việc thứ 24
Success!!!
Công việc thứ 25
Success!!!
Công việc thứ 26
Success!!!
Công việc thứ 27
Success!!!
Công việc thứ 28
Success!!!
Công việc thứ 29
Success!!!
Công việc thứ 30
Success!!!
Công việc thứ 31
Success!!!
Công việc thứ 32
Success!!!
Công việc thứ 33
Success!!!
Công việc thứ 34
Success!!!
Công việc thứ 35
Success!!!
Cô

In [16]:
raw_job_details.sample(5)

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Thời gian thử việc,Cấp bậc,Yêu cầu giới tính,Số lượng tuyển,Hình thức làm việc,...,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
935,https://vieclam24h.vn/ke-toan/ke-toan-tong-hop...,Công Ty Cổ Phần Dược Phẩm Cmi,Kế Toán Tổng Hợp,10 - 15 triệu,Hà Nội,2 tháng,Chuyên viên- nhân viên,None,1,Toàn thời gian cố định,...,Đại học,3 năm,Kế toán/Kiểm toán/Nhân sự,"Hà Nội, Số 17 Sơn Tây, Ba Đình",- Phụ trách tất cả các công việc liên q...,- Độ tuổi dưới 40 tuổi- Tốt nghi...,- Thời gian thử việc: 2 tháng- S...,sơnkế toántoánViệc làm Hà NộiKế toánCông Ty Cổ...,số 17 phố Sơn Tây - Phường Điện Biên - Quận Ba...,10 - 150 người
1120,https://vieclam24h.vn/cham-soc-khach-hang/nhan...,Công Ty TNHH TM Hồng Ký Mì Gia,Nhân Viên Content Creator (Marketing),8 - 12 triệu,TP.HCM,None,Chuyên viên- nhân viên,None,2,Toàn thời gian cố định,...,Trung cấp,1 năm,Chăm sóc khách hàng/Marketing/Hành chính - Thư ký,"TP.HCM, Số 2 Đường số 4, P.An Lạc A, Bình Tân","- Trực tiếp sáng tạo, lên kịch bản, kết hợp te...","- Có 1 năm ở vị trí tương đương- Sáng tạo, hiể...",- Mức lương bao gồm: Lương cơ bản+ phụ cấp+ Th...,contenthuyện bình tânquận bình tânViệc làm TP....,"Số 2 đường số 4, P.An Lạc A. Q.Bình Tân, TP. HCM",10 - 150 người
194,https://vieclam24h.vn/khoa-hoc-ky-thuat/truong...,Công Ty Cổ Phần Sản Xuất Vật Liệu Xây Dựng Min...,Trưởng Phòng Kỹ Thuật Nghiên Cứu Phát Triển Sả...,12 - 20 triệu,TP.HCM,None,Quản lý cấp trung,None,1,Toàn thời gian cố định,...,Cao đẳng,2 năm,Khoa học - Kỹ thuật/Xây dựng/Quản lý tiêu chuẩ...,"TP.HCM, 652/37 A Quốc Lộ 13, P Hiệp Bình Phước...","NGÀNH NGHỀ:VLXD mới: keo dán gạch, keo chà ron...",Tiếng Anh giao tiếp cơ bảnCó kinh nghiệm 2 đến...,"Lương thưởng theo năng lực, thương lượngMôi t...",Việc làm TP.HCMKhoa học - Kỹ thuậtCông Ty Cổ P...,"652/37 A Quốc Lộ 13, P Hiệp Bình Phước, Thủ Đức",Dưới 10 người
330,https://vieclam24h.vn/khoa-hoc-ky-thuat/ky-thu...,Công Ty Cổ Phần Thành Thành Công - Biên Hòa,Kỹ Thuật Viên Bảo Trì Tự Động ( Lương 10-15 Tr...,10 - 15 triệu,Tây Ninh,None,Chuyên viên- nhân viên,Nam,2,Toàn thời gian cố định,...,Đại học,2 năm,Khoa học - Kỹ thuật/Vận hành - Bảo trì - Bảo d...,"Tây Ninh, Tân Hưng , Tân Châu","- Kiểm tra, bảo trì và cân chỉnh các thiết bị,...",- Tốt nghiệp Đại học chuyên ngành: Điện tự độn...,- Mức lương: thỏa thuận theo năng lực- Được Cô...,Việc làm Tây NinhKhoa học - Kỹ thuậtCông Ty Cổ...,Tân Hưng - Huyện Tân Châu - Tỉnh Tây Ninh,Trên 300 người
538,https://vieclam24h.vn/nong-lam-ngu-nghiep/nhan...,Công Ty Cổ Phần Xây Lắp Vật Tư Kỹ Thuật,Nhân Viên Quản Lý Tổ Chăm Sóc Cây Xanh Tại Nha...,7 - 9 triệu,"Khánh Hòa, Hà Nội",None,Chuyên viên- nhân viên,None,2,Toàn thời gian cố định,...,Trung cấp,1 năm,Nông - Lâm - Ngư nghiệp/Xây dựng/Kiến trúc - T...,"Khánh Hòa, Nha Trang; Hà Nội, Ba Đình",- Xây dựng kế hoạch chăm sóc cây cảnh định kỳ ...,- Tốt nghiệp từ Trung cấp trở lên chuyên ngành...,- Được hưởng mức thu nhập tương xứng với năng ...,Việc làm Khánh HòaNông - Lâm - Ngư nghiệpCông ...,"Số 534, phố Minh Khai, Phường Vĩnh Tuy, Quận H...",150 - 300 người


In [17]:
# Lưu vào file craw_dataset.csv
raw_job_details.info()
# raw_job_details.to_csv('../data/raw_dataset.csv', encoding='utf-8-sig', index=False)
raw_job_details.to_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/raw_dataset.csv', encoding='utf-8-sig', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1877 entries, 0 to 1876
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Liên kết             1877 non-null   object
 1   Tên công ty          1877 non-null   object
 2   Tên công việc        1877 non-null   object
 3   Mức lương            1877 non-null   object
 4   Khu vực tuyển        1877 non-null   object
 5   Thời gian thử việc   993 non-null    object
 6   Cấp bậc              1877 non-null   object
 7   Yêu cầu giới tính    516 non-null    object
 8   Số lượng tuyển       1877 non-null   object
 9   Hình thức làm việc   1877 non-null   object
 10  Độ tuổi              812 non-null    object
 11  Yêu cầu bằng cấp     1639 non-null   object
 12  Yêu cầu kinh nghiệm  1877 non-null   object
 13  Ngành nghề           1877 non-null   object
 14  Địa điểm làm việc    1876 non-null   object
 15  Mô tả công việc      1877 non-null   object
 16  Yêu cầ

# Một số phần demo để testing
- Đây là đoạn code để người collect data có thể test thử trước khi chạy toàn bộ code

In [18]:
# Demo lất page content
url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6'
page_content = get_content(url, headers)
page_text = page_content.text
soup = BeautifulSoup(page_text, 'html.parser')
buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
buffer2 = buffer1.find('span',class_="font-semibold").text
n_jobs = int(buffer2.replace(',',''))
print('Có {} công việc'.format(n_jobs))
n_pages = min(51, math.ceil(n_jobs/30) + 1)
print('Có {} trang'.format(n_pages-1))

Success!!!
Có 101 công việc
Có 4 trang


In [19]:
# Demo lất page content

url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6'
page_content = get_content(url, headers)
page_text = page_content.text

job_pool = pd.DataFrame(columns=['Liên kết', 'Tên công việc','Tên công ty','Mức lương','Khu vực tuyển'])

soup = BeautifulSoup(page_text, 'html.parser')
containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
print('Đã tìm thấy {} công việc'.format(len(containers)))
for container in containers:
    dict_job = {}

    buffer = BASE_URL + container.find('a', href=True)['href']
    dict_job['Liên kết'] = buffer if buffer else None

    buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
    dict_job['Tên công việc'] = buffer.text if buffer else None

    buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
    dict_job['Tên công ty'] = buffer.text if buffer else None

    buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
    dict_job['Mức lương'] = buffer.text if buffer else None

    buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None

    job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool

Success!!!
Đã tìm thấy 30 công việc


,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
0,https://vieclam24h.vn/bien-phien-dich/thuc-tap...,Thực Tập Sinh Biên Dịch Tiếng Anh - Lương 5Tr ...,Công Ty TNHH Mediastep Software Việt Nam,5 - 6 triệu,TP.HCM
1,https://vieclam24h.vn/thiet-ke-sang-tao-nghe-t...,Thực Tập Sinh Thiết Kế Đồ Họa,Công Ty TNHH Miligo,1 - 5 triệu,TP.HCM
2,https://vieclam24h.vn/bat-dong-san/thuc-tap-si...,Thực Tập Sinh Kinh Doanh Tại Thanh Hóa,Công Ty Cp Đầu Tư Và Kinh Doanh Bất Động Sản S...,5 - 10 triệu,Thanh Hóa
3,https://vieclam24h.vn/nhan-su/thuc-tap-sinh-nh...,Thực Tập Sinh Nhân Sự (C&B),Công Ty CP Pt Kinh Doanh Thịnh Phát,2.5 - 3 triệu,Hà Nội
4,https://vieclam24h.vn/marketing/thuc-tap-sinh-...,Thực Tập Sinh Marketing,Công Ty Cổ Phần Khoa Học Công Nghệ Xanh,1 - 3 triệu,Hà Nội
5,https://vieclam24h.vn/it-phan-mem/thuc-tap-sin...,Thực Tập Sinh Lập Trình PHP,Công Ty Cổ Phần Vinasimex,10 - 30 triệu,"Hà Nội, TP.HCM"
6,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Thực Tập Sinh Kế Toán,Công Ty TNHH Sôcôla Marou,1 - 3 triệu,TP.HCM
7,https://vieclam24h.vn/ban-hang-kinh-doanh/thuc...,Thực Tập Sinh Trình Dược Viên,Công Ty Cổ Phần Đất Dược,2 - 9 triệu,"TP.HCM, Bình Dương, Cần Thơ, Đồng Nai"
8,https://vieclam24h.vn/ban-hang-kinh-doanh/q1-h...,[Q1 - HCM] Thực Tập Sinh Phòng Kinh Doanh B2B,Cty TNHH Phòng Khám Đa Khoa Jio Health,3 - 3.5 triệu,TP.HCM
9,https://vieclam24h.vn/thuc-tap-sinh/thuc-tap-s...,Thực Tập Sinh Kinh Doanh Thiết Bị Ngành Kỹ Thuật,Công Ty Cổ Phần Tự Động Việt,2 - 8 triệu,TP.HCM


In [20]:

url = 'https://vieclam24h.vn/ban-hang/cong-tac-vien-ban-hang-online-my-pham-thuc-pham-chuc-nang-c113p121id200062770.html'
job_content = get_content(url, headers=headers).text
soup = BeautifulSoup(job_content, 'html.parser')

Success!!!


In [10]:
# Demo lấy chi tiết

raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển',
# Thông tin chung
'Thời gian thử việc','Cấp bậc', 'Yêu cầu giới tính','Số lượng tuyển', 'Hình thức làm việc','Độ tuổi', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])

dict_job = {}

dict_job['Liên kết'] = url

overview = soup.find('div', class_='md:ml-7 w-full')
try:
    buffer = overview.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
    dict_job['Tên công ty'] = buffer.text if buffer else None
except:
    dict_job['Tên công ty'] = None
try:
    buffer = overview.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
    dict_job['Tên công việc'] = buffer.text if buffer else None
except:
    dict_job['Tên công việc'] = None
try:
    buffer = overview.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
    dict_job['Mức lương'] = buffer.text if buffer else None
except:
    dict_job['Mức lương'] = None

try:
    buffer = overview.find_all('a', class_ ="hover:text-se-accent")
    dict_job['Khu vực tuyển'] = ' '.join([element.text for element in buffer]) if buffer else None
except:
    dict_job['Khu vực tuyển'] = None

try:
    buffer = soup.find('p', string="Yêu cầu giới tính")
    dict_job['Yêu cầu giới tính'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Yêu cầu giới tính'] = None

try:
    buffer = soup.find('p', string="Cấp bậc")
    dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Cấp bậc'] = None

try:
    buffer = soup.find('p', string="Thời gian thử việc")
    dict_job['Thời gian thử việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Thời gian thử việc'] = None

try:
    buffer = soup.find('p', string="Số lượng tuyển")
    dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Số lượng tuyển'] = None


try:
    buffer = soup.find('p', string="Hình thức làm việc")
    dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Hình thức làm việc'] = None

try:
    buffer = soup.find('p', string="Độ tuổi")
    dict_job['Độ tuổi'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Độ tuổi'] = None

try:
    buffer = soup.find('p', string="Yêu cầu bằng cấp")
    dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Yêu cầu bằng cấp'] = None

try:
    buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
    dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Yêu cầu kinh nghiệm'] = None

try:
    buffer = soup.find('p', string="Ngành nghề")
    dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Ngành nghề'] = None

try:
    buffer1 = soup.find('h4', string="Mô tả công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None
except:
    dict_job['Mô tả công việc'] = None

try:
    buffer1 = soup.find('h4', string="Yêu cầu công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None
except:
    dict_job['Yêu cầu công việc'] = None

try:
    buffer1 = soup.find('h4', string="Quyền lợi")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Quyền lợi'] = buffer2.text if buffer2 else None
except:
    dict_job['Quyền lợi'] = None

try:
    buffer1 = soup.find('h4', string="Địa điểm làm việc")
    buffer2 = buffer1.find_next_sibling('ul')
    buffer3 = buffer2.find_all('li', class_ = "jsx-d84db6a84feb175e mb-2 flex text-14")
    dict_job['Địa điểm làm việc'] = '; '.join([element.text for element in buffer3 if element.text.strip()]) if buffer3 else None
except:
    dict_job['Địa điểm làm việc'] = None

try:
    buffer1 = soup.find('h3', string="Từ khoá")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Từ khóa'] = buffer2.text if buffer2 else None
except:
    dict_job['Từ khóa'] = None

company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
try:
    for buffer in buffers:
        if buffer.get_text() == 'Địa chỉ:':
            dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
        if buffer.get_text() == 'Quy mô:':
            dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None
except:
    dict_job['Địa chỉ công ty'] = None
    dict_job['Quy mô công ty'] = None

raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)
raw_job_details


0    25 - 29 tuổi
Name: Độ tuổi, dtype: object